In [ ]:
import osmium as o
import subprocess

import sys

In [ ]:
class OSMHandler(o.SimpleHandler):
    def __init__(self):
        o.SimpleHandler.__init__(self)
        self.ways_elements = {'elements':[]}
        self.relation_elements = {'elements':[]}
        self.ways_position = {}
        self.relations_position = {}
        self.nodes = []
        self.ways = []
        self.relations = []
        self.wkbfab = o.geom.WKBFactory()
    
    def node(self, n):
        pass

    def way(self, w):
        nodes_ids = []
        geometry = []
        bounds = {
            'minlat': None,
            'minlon': None,
            'maxlat': None, 
            'maxlon': None
        }
        tags = {}

        for elem in w.nodes:
            nodes_ids.append(elem.ref)

            geometry.append({
                'lat': elem.lat,
                'lon': elem.lon
            })

            if(bounds['minlat'] == None):
                bounds['minlat'] = elem.lat
            elif(elem.lat < bounds['minlat']):
                    bounds['minlat'] = elem.lat

            if(bounds['minlon'] == None):
                bounds['minlon'] = elem.lon
            elif(elem.lon < bounds['minlon']):
                    bounds['minlon'] = elem.lon

            if(bounds['maxlat'] == None):
                bounds['maxlat'] = elem.lat
            elif(elem.lat > bounds['maxlat']):
                    bounds['maxlat'] = elem.lat

            if(bounds['maxlon'] == None):
                bounds['maxlon'] = elem.lon
            elif(elem.lon > bounds['maxlon']):
                    bounds['maxlon'] = elem.lon

        for tag in w.tags:
            tags[tag.k] = tag.v

        self.ways_elements['elements'].append({
            'type': 'way',
            'id': w.id,
            'bounds': bounds,
            'nodes': nodes_ids,
            'geometry': geometry,
            'tags': tags
        })

        self.ways_position[w.id] = len(self.ways_elements['elements'])-1

    def relation(self, r):
        members = []
        tags = {}

        for tag in r.tags:
            tags[tag.k] = tag.v

        for member in r.members:
            type = member.type

            if type == 'w':
                type = 'way'

            members.append({
                'id': member.ref,
                'type': type,
                'role': member.role,
                'geometry': []
            })

        self.relation_elements['elements'].append({
            'type': 'relation',
            'id': r.id,
            'members': members,
            'bounds': None,
            'tags': tags
        })

        self.relations_position[r.id] = len(self.relation_elements['elements'])-1



In [ ]:
def get_osm_filter(layer_type):

    '''
        Create an osm filter based on urban-toolkit layer types (building, road, coastline, water, parks)

        Args:
            layer_type (string): Name of the layer to generate filters for. Possible values: 'building', 'road', 'coastline', 'water', 'parks'

        Returns:
            result (object): An object containing all the filters for way, rel
    '''

    filters = {}
    filters['way'] = []
    filters['node'] = []
    filters['rel'] = []

    if layer_type == 'water':
        natural_types = ['water', 'wetland', 'bay', 'strait', 'spring']
        water_types = ['pond', 'reservoir', 'lagoon', 'stream_pool', 'lake', 'pool', 'canal', 'river']
        # natural_types = ['water']
        
        filters['way'].extend(['["natural"="%s"]'%t for t in natural_types])
        filters['rel'].extend(['["natural"="%s"]'%t for t in natural_types])
        filters['way'].extend(['["water"="%s"]'%t for t in water_types])
        filters['rel'].extend(['["water"="%s"]'%t for t in water_types])

    if layer_type == 'parks':
        natural_types = ['wood', 'grass']
        land_use_types = ['wood', 'grass', 'forest', 'orchad', 'village_green', 'vineyard', 'cemetery', 'meadow', 'village_green']
        leisure_types = ['dog_park', 'park', 'playground', 'recreation_ground']

        filters['way'].extend(['["natural"="%s"]'%t for t in natural_types])
        filters['rel'].extend(['["natural"="%s"]'%t for t in natural_types])
        filters['way'].extend(['["landuse"="%s"]'%t for t in land_use_types])
        filters['rel'].extend(['["landuse"="%s"]'%t for t in land_use_types])
        filters['way'].extend(['["leisure"="%s"]'%t for t in leisure_types])
        filters['rel'].extend(['["leisure"="%s"]'%t for t in leisure_types])

    if layer_type == 'roads':
        filters['way'].extend(['["highway"]["area"!~"yes"]["highway"!~"cycleway|footway|proposed|construction|abandoned|platform|raceway"]'])
        # filters['way'].extend(['"area"!~"yes"'])
        # filters['way'].extend(['"highway"!~"proposed|construction|abandoned|platform|raceway)"'])
        # filters['rel'].extend(['"highway"="%s"'%t for t in highway_types])

    if layer_type == 'coastline':
        filters['way'].extend(['["natural"="coastline"]'])

    if layer_type == 'buildings':
        filters['way'].extend(['["building"]'])
        filters['way'].extend(['["building:part"]'])
        filters['way'].extend(['["type"="building"]'])
        # filters['way'].extend(['["building:levels"]'])
        # filters['way'].extend(['["building:min_level"]'])
        # filters['way'].extend(['["building:height"]'])
        # filters['way'].extend(['["roof:levels"]'])
        # filters['way'].extend(['["roof:height"]'])
        # filters['way'].extend(['["roof:shape"]'])
        # filters['rel'].extend(['["building"]'])
        # filters['rel'].extend(['["building:levels"]'])
        # filters['rel'].extend(['["building:min_level"]'])
        # filters['rel'].extend(['["building:height"]'])
        # filters['rel'].extend(['["roof:levels"]'])
        # filters['rel'].extend(['["roof:height"]'])
        # filters['rel'].extend(['["roof:shape"]'])

    return filters

In [ ]:
def fill_relation_geom(ways_elements, relation_elements, ways_position, relations_position):

    def get_bounds(geometry):
        bounds = {
            'minlat': None,
            'minlon': None,
            'maxlat': None, 
            'maxlon': None
        }

        for elem in geometry:

            if(bounds['minlat'] == None):
                bounds['minlat'] = elem['lat']
            elif(elem['lat'] < bounds['minlat']):
                    bounds['minlat'] = elem['lat']

            if(bounds['minlon'] == None):
                bounds['minlon'] = elem['lon']
            elif(elem['lon'] < bounds['minlon']):
                    bounds['minlon'] = elem['lon']

            if(bounds['maxlat'] == None):
                bounds['maxlat'] = elem['lat']
            elif(elem['lat'] > bounds['maxlat']):
                    bounds['maxlat'] = elem['lat']

            if(bounds['maxlon'] == None):
                bounds['maxlon'] = elem['lon']
            elif(elem['lon'] > bounds['maxlon']):
                    bounds['maxlon'] = elem['lon']
        

    for relation_id in relations_position:
        relation = relation_elements['elements'][relations_position[relation_id]]

        for member in relation['members']:
            if(member['type'] == 'way' and member['id'] in ways_position): # TODO: verify why some ways that compose certain relations do not appear in the ways array
                way = ways_elements['elements'][ways_position[member['id']]]

                relation['bounds'] = get_bounds(way['geometry'])
                relation['geometry'] = way['geometry'].copy()
                

In [ ]:
osmhandler = OSMHandler()
osmhandler.apply_file("greenland-latest.osm.pbf", locations=True)

ways_elements = osmhandler.ways_elements
relation_elements = osmhandler.relation_elements
ways_position = osmhandler.ways_position
relations_position = osmhandler.relations_position

fill_relation_geom(ways_elements, relation_elements, ways_position, relations_position)

osm_elements = {'elements': ways_elements['elements'] + relation_elements['elements']}

# apt get install osmium-tool
proc = subprocess.call(['wsl', 'osmium', 'extract', '-b', aux, '-o', filename, '--overwrite', input_filename], shell=True) # TODO: make it not depend on the OS

# with open('output.txt', 'w') as f:
#     f.write(str(relation_elements))